In [1]:
#Importing necessary libraries
import pandas as pd 
import numpy as np

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException,WebDriverException, ElementClickInterceptedException, TimeoutException,ElementNotInteractableException,NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import requests
from bs4 import BeautifulSoup as bs
import concurrent.futures
import time

import warnings 
warnings.filterwarnings("ignore")

# Scraping data from www.cardekho.com

In [56]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
            'Accept-Language': 'en-US, en;q=0.5'})

city= 'Chennai'
driver= webdriver.Chrome('chromedriver.exe')
driver.get('https://www.cardekho.com/used-cars+in+chennai')

html = driver.find_element(By.TAG_NAME,'html')
for i in range(0,200):    
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

In [57]:
data= driver.find_elements(By.XPATH,"//div[@class=' gsc_row minheight']")
links=[]
for i in data:
    try:
        link_data= i.find_element(By.TAG_NAME,'a')
        link= link_data.get_attribute('href')
        links.append(link)
    except NoSuchElementException:
        continue    
len(links)

907

In [58]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
            'Accept-Language': 'en-US, en;q=0.5'})

year=[]
brand=[]
model=[]
fuel=[]
trnsmsn=[]
engine_disp=[]
km_driven=[]
owners=[]
location=[]
price=[]

def get_data1(soup):

    try:
        yr_model_data= soup.find('h1').text
        year_data= yr_model_data[0:4]
        year.append(year_data)
        model_data=yr_model_data[4:].strip()
        model.append(model_data)
        brand_data= model_data.split(' ')[0]
        brand.append(brand_data)
    except AttributeError:
        year.append(np.nan)
        model.append(np.nan)
        brand.append(np.nan)
        
    try:
        price_data= soup.find('div',class_='priceSection').text.strip().split('₹ ')[1]
        price.append(price_data)
    except AttributeError:
        price.append(np.nan)

    try:
        #table_data= soup.find('div',class_='white-bg VDPtopCard overviewCArd').text
        table_data= soup.find('ul',class_='gsc_row detailsList').text
        fuel_check1= table_data.find('Petrol')
        fuel_check2= table_data.find('Diesel')
        if fuel_check1!=(-1):
            fuel.append('Petrol')
        elif fuel_check2!=(-1):
            fuel.append('Diesel')
        else:
            fuel.append(np.nan)
            
        transmission_check1= table_data.find('Manual')
        transmission_check2= table_data.find('Automatic')
        if fuel_check1!=(-1):
            trnsmsn.append('Manual')
        elif fuel_check2!=(-1):
            trnsmsn.append('Automatic')
        else:
            trnsmsn.append(np.nan)
    except AttributeError:
        fuel.append(np.nan)
        
    try:
        kms_driven_data= table_data.split('KMs Driven')[1].split(' ')[0]
        km_driven.append(kms_driven_data)
    except IndexError:
        km_driven.append(np.nan)
    except AttributeError:
        km_driven.append(np.nan)
    
    try:
        owners_data= table_data.split('No Of Owner(s)')[1].split(' ')[0]
        owners.append(owners_data)
    except AttributeError:
        owners.append(np.nan)
    except IndexError:
        owners.append(np.nan)
        
    try:
        disp_data= table_data.split('Engine Displacement')[1].split(' ')[0]
        engine_disp.append(disp_data)
    except IndexError:
        engine_disp.append(np.nan)    
    location.append(city)
    
    
def get_data2(soup):
    
    try:
        yr_model_data= soup.find('div',class_='vdp-head').text
        year_data= yr_model_data[0:4]
        year.append(year_data)
        model_data=yr_model_data[4:].strip()
        model.append(model_data)
        brand_data= model_data.split(' ')[0]
        brand.append(brand_data)
    except AttributeError:
        year.append(np.nan)
        model.append(np.nan)
        brand.append(np.nan)

    try:
        price_data= soup.find('span',class_='amount').text
        price.append(price_data)
    except AttributeError:
        price.append(np.nan)
    

    try:
        table_data= soup.find('div',class_='GenDetailBox').text
        fuel_check1= table_data.find('Petrol')
        fuel_check2= table_data.find('Diesel')
        if fuel_check1!=(-1):
            fuel.append('Petrol')
        elif fuel_check2!=(-1):
            fuel.append('Diesel')
        else:
            fuel.append(np.nan)

        try:
            kms_driven_data= table_data.split('Kms')[0].strip().split('Dealer')[1]
            km_driven.append(kms_driven_data)
        except IndexError:
            try:
                kms_driven_data= table_data.split('Kms')[0].strip().split('Individual')[1]
                km_driven.append(kms_driven_data)
            except:
                km_driven.append(np.nan)

        try:
            owners_data= table_data.split('Owner')[0].split('Kms')[1].strip()
            owners.append(owners_data)
        except IndexError:
            owners.append(np.nan)

        transmission_check1= table_data.find('Manual')
        transmission_check2= table_data.find('Automatic')
        if fuel_check1!=(-1):
            trnsmsn.append('Manual')
        elif fuel_check2!=(-1):
            trnsmsn.append('Automatic')
        else:
            trnsmsn.append(np.nan)
    except:
        fuel.append(np.nan)
        km_driven.append(np.nan)
        owners.append(np.nan)
        trnsmsn.append(np.nan)
        
    try:
        table2_data= soup.find('div',class_='SpecsFeatureList').text
        disp_data= table2_data.split('Engine')[1].split(' ')[0]
        engine_disp.append(disp_data)
    except:
        engine_disp.append(np.nan)

    location.append(city)
    

def scrap_initiate(url):
    r= requests.get(url,headers=HEADERS)
    soup_contents= bs(r.content)
    try:
        table_data= soup_contents.find('div',class_='white-bg VDPtopCard overviewCArd').text
        get_data1(soup_contents)
    except AttributeError:
        get_data2(soup_contents)
        
start_time = time.time() 
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(scrap_initiate,links)
    
total_time = time.time() - start_time
        
results_df=pd.DataFrame({'Year':year,
                         'Brand':brand,
                         'Model':model,
                         'Fuel Type':fuel,
                         'Transmission':trnsmsn,
                         'Engine Capacity':engine_disp,
                         'Kms Driven':km_driven,
                         'No. of Owners':owners,
                         'Location':location,
                         'Price':price})

print('Execution Time:',round(total_time,2))
driver.close()
results_df

Execution Time: 101.08


,Year,Brand,Model,Fuel Type,Transmission,Engine Capacity,Kms Driven,No. of Owners,Location,Price
0,2016,Mahindra,Mahindra TUV 300,Diesel,Automatic,1493,"1,10,666",1st,Chennai,"5,98,000"
1,2017,Hyundai,Hyundai Grand i10,Petrol,Manual,1197,"41,122",1st,Chennai,"4,96,000"
2,2018,Hyundai,Hyundai Verna,Petrol,Manual,1591,"47,307",1st,Chennai,"8,70,000"
3,2017,Hyundai,Hyundai Creta,Diesel,Automatic,1582,"49,848",1st,Chennai,"11,02,000"
4,2020,Maruti,Maruti Swift Dzire,Petrol,Manual,1197,"33,626",1st,Chennai,"6,71,000"
...,...,...,...,...,...,...,...,...,...,...
902,2018,Honda,Honda Jazz 1.2 S I Vtec,Petrol,Manual,1199,"4,442",First,Chennai,7.75 Lakh*
903,2007,Maruti,Maruti Alto 800 Lxi,Petrol,Manual,796,"1,00,050",Second,Chennai,1.20 Lakh*
904,2008,Maruti,Maruti Zen Estilo 1.1 Lxi Bsiii,Petrol,Manual,1061,"50,000",First,Chennai,1.20 Lakh*
905,2005,Toyota,Toyota Innova 2.5 G (diesel) 8 Seater Bs Iv,Diesel,Automatic,2494,"2,35,000",Fourth,Chennai,3 Lakh*


In [60]:
results_df.to_csv(f'Results_{city}.csv')

In [61]:
results_df.isnull().sum()

Year               0
Brand              0
Model              0
Fuel Type          2
Transmission       2
Engine Capacity    2
Kms Driven         0
No. of Owners      0
Location           0
Price              0
dtype: int64

# Scraping data from www.cars24.com

In [8]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
            'Accept-Language': 'en-US, en;q=0.5'})

city= 'Noida'
driver= webdriver.Chrome('chromedriver.exe')
driver.get('https://www.cars24.com/buy-used-car?sort=P&storeCityId=134&pinId=201301')

html = driver.find_element(By.TAG_NAME,'html')
for i in range(0,200):    
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

In [9]:
data= driver.find_elements(By.XPATH,"//div[@class='col-4']")
links=[]
for i in data:
    try:
        link_data=i.find_element(By.TAG_NAME,'a')
        link= link_data.get_attribute('href')
        links.append(link)
    except NoSuchElementException:
        continue
len(links)

667

In [10]:
year=[]
brand=[]
model=[]
fuel=[]
trnsmsn=[]
engine_disp=[]
km_driven=[]
owners=[]
location=[]
price=[]

def scrap_cars24(url):
    r= requests.get(url,headers=HEADERS)
    soup= bs(r.content)
    
    try:
        data= soup.find('h2',class_='_2geSF').text.strip()
        components= data.split(' ')
        year_data= components[0]
        year.append(year_data)
        brand_data= components[1]
        brand.append(brand_data)
        trnsmsn_data= components[-1]
        trnsmsn.append(trnsmsn_data)
        model_data= data[5:]
        model.append(model_data.replace(trnsmsn_data,'').strip())
    except AttributeError:
        year.append(np.nan)
        brand.append(np.nan)
        trnsmsn.append(np.nan)
        model.append(np.nan)
        
    try:
        table_data= soup.find('div',class_='DB7s9').text
        kms_driven_data= table_data.split('Kilometers Driven')[1].split(' ')[0]
        km_driven.append(kms_driven_data)
        fuel_data= table_data.split('Fuel Type')[1].split(' ')[0]
        fuel.append(fuel_data)
        owner_data= table_data.split('Owner')[1].split(' ')[0]
        owners.append(owner_data)
    except AttributeError:
        km_driven.append(np.nan)
        fuel.append(np.nan)
        owners.append(np.nan)

    try:
        price_data= soup.find('strong',class_='_2yYvS Wqfj6').text.strip().split('₹')
        price.append(price_data[-1])
    except AttributeError:
        price.append(np.nan)
    
    engine_disp.append(np.nan)
    location.append(city)
    
start_time = time.time() 
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(scrap_cars24,links)
    
total_time = time.time() - start_time
    
results_df=pd.DataFrame({'Year':year,
                         'Brand':brand,
                         'Model':model,
                         'Fuel Type':fuel,
                         'Transmission':trnsmsn,
                         'Engine Capacity':engine_disp,
                         'Kms Driven':km_driven,
                         'No. of Owners':owners,
                         'Location':location,
                         'Price':price})

print('Execution Time:',round(total_time,2))
driver.close()
results_df

Execution Time: 87.39


,Year,Brand,Model,Fuel Type,Transmission,Engine Capacity,Kms Driven,No. of Owners,Location,Price
0,2011,Hyundai,Hyundai i10 MAGNA 1.2 KAPPA2,Petrol,MANUAL,NaN,"72,229",1st,Noida,"2,82,899"
1,2009,Hyundai,Hyundai i10 ERA 1.1 IRDE,Petrol,MANUAL,NaN,"44,327",1st,Noida,"1,99,699"
2,2020,Hyundai,Hyundai Grand i10 MAGNA 1.2 VTVT,Petrol,MANUAL,NaN,"6,470",1st,Noida,"6,08,399"
3,2016,Mahindra,Mahindra Kuv100 K8 6 STR,Petrol,MANUAL,NaN,"19,930",1st,Noida,"4,60,899"
4,2020,Maruti,Maruti IGNIS SIGMA 1.2 K12,Petrol,MANUAL,NaN,"15,199",1st,Noida,"5,06,599"
...,...,...,...,...,...,...,...,...,...,...
662,2019,Tata,Tata NEXON XM 1.2,Petrol,MANUAL,NaN,"28,061",2nd,Noida,"6,68,899"
663,2012,Skoda,Skoda Rapid AMBITION 1.6 MPFI MT,Petrol,MANUAL,NaN,"85,668",1st,Noida,"2,74,699"
664,2020,Maruti,Maruti Celerio ZXI AMT,Petrol,AUTOMATIC,NaN,"13,152",1st,Noida,"5,79,099"
665,2018,Maruti,Maruti Swift VXI,Petrol,MANUAL,NaN,"23,476",2nd,Noida,"5,60,599"


In [12]:
results_df.to_csv(f'Results(cars24.com)_{city}.csv')

In [13]:
results_df.isnull().sum()

Year                 0
Brand                0
Model                0
Fuel Type            0
Transmission         0
Engine Capacity    667
Kms Driven           0
No. of Owners        0
Location             0
Price                0
dtype: int64

# Creating the final dataset

In [109]:
df1= pd.read_csv('Results_Ahmedabad.csv')
df2= pd.read_csv('Results_Bangalore.csv')
df3= pd.read_csv('Results_Chennai.csv')
df4= pd.read_csv('Results_Hyderabad.csv')
df5= pd.read_csv('Results_Kolkata.csv')
df6= pd.read_csv('Results_Mumbai.csv')
df6_1=pd.read_csv('Results_Mumbai_1.csv')
df6_2=pd.read_csv('Results_Mumbai_2.csv')
df7= pd.read_csv('Results_New_Delhi.csv')
df7_1= pd.read_csv('Results_New_Delhi_1.csv')
df7_2= pd.read_csv('Results_New_Delhi_2.csv')
df7_3= pd.read_csv('Results_New_Delhi_3.csv')
df8= pd.read_csv('Results_Pune.csv')
df9= pd.read_csv('Results_Gurgaon.csv')
df10= pd.read_csv('Results_Noida.csv')
df11= pd.read_csv('Results_Jaipur.csv')

df_1= pd.read_csv('Results(cars24.com)_Ahmedabad.csv')
df_2= pd.read_csv('Results(cars24.com)_Bangalore.csv')
df_3= pd.read_csv('Results(cars24.com)_Chennai.csv')
df_4= pd.read_csv('Results(cars24.com)_Hyderabad.csv')
df_5= pd.read_csv('Results(cars24.com)_Kolkata.csv')
df_6= pd.read_csv('Results(cars24.com)_Mumbai.csv')
df_7= pd.read_csv('Results(cars24.com)_New_Delhi.csv')
df_8= pd.read_csv('Results(cars24.com)_Pune.csv')
df_9= pd.read_csv('Results(cars24.com)_Gurgaon.csv')
df_10= pd.read_csv('Results(cars24.com)_Noida.csv')

In [110]:
data= [df1,df2,df3,df4,df5,df6,df6_1,df6_2,df7,df7_1,df7_2,df7_3,df8,df9,df10,df11,
       df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10]
df= pd.concat(data,axis=0)
df.reset_index(inplace=True)
df

,index,Unnamed: 0,Year,Brand,Model,Fuel Type,Transmission,Engine Capacity,Kms Driven,No. of Owners,Location,Price
0,0,0,2016.0,Maruti,Maruti Ciaz,Petrol,Manual,1373.0,"36,534",1st,Ahmedabad,"6,12,000"
1,1,1,2020.0,Datsun,Datsun RediGO,Petrol,Manual,799.0,"23,840",1st,Ahmedabad,"3,81,000"
2,2,2,2020.0,Honda,Honda City,Petrol,Manual,1498.0,"38,749",1st,Ahmedabad,"9,90,000"
3,3,3,2016.0,Honda,Honda City,Petrol,Manual,1497.0,"25,542",1st,Ahmedabad,"7,33,000"
4,4,4,2017.0,Hyundai,Hyundai Verna,Petrol,Manual,1591.0,"60,238",1st,Ahmedabad,"8,93,000"
...,...,...,...,...,...,...,...,...,...,...,...,...
20907,662,662,2019.0,Tata,Tata NEXON XM 1.2,Petrol,MANUAL,NaN,"28,061",2nd,Noida,"6,68,899"
20908,663,663,2012.0,Skoda,Skoda Rapid AMBITION 1.6 MPFI MT,Petrol,MANUAL,NaN,"85,668",1st,Noida,"2,74,699"
20909,664,664,2020.0,Maruti,Maruti Celerio ZXI AMT,Petrol,AUTOMATIC,NaN,"13,152",1st,Noida,"5,79,099"
20910,665,665,2018.0,Maruti,Maruti Swift VXI,Petrol,MANUAL,NaN,"23,476",2nd,Noida,"5,60,599"


In [111]:
df.drop(columns=['index','Unnamed: 0'],inplace=True)

In [112]:
df.isnull().sum()

Year                  1
Brand                 1
Model                 1
Fuel Type           415
Transmission        415
Engine Capacity    4895
Kms Driven            1
No. of Owners         1
Location              0
Price                 1
dtype: int64

In [113]:
df= df[df['Price'].notna()]
df.reset_index(inplace=True,drop=True)

In [114]:
df['Price'][11:16]

11    1.80 Crore*
12      15,53,000
13       6,12,000
14       8,20,000
15    1.25 Crore*
Name: Price, dtype: object

In [115]:
price_list=[]
for i in range(len(df['Price'])):
    value=df['Price'][i].replace(',','')
    value=value.replace('*','')
    data_check1= df['Price'][i].find('Lakh')
    data_check2= df['Price'][i].find('Crore')
    if data_check1!=(-1):
        value=value.replace(' Lakh','')
        value=float(value)*100000
    elif data_check2!=(-1):
        value=value.replace(' Crore','')
        value=float(value)*10000000
    price_list.append(value)
price_list[11:16]

[18000000.0, '1553000', '612000', '820000', 12500000.0]

In [116]:
df['Price']= pd.to_numeric(price_list)

In [117]:
df= df.sample(frac=1)
df

,Year,Brand,Model,Fuel Type,Transmission,Engine Capacity,Kms Driven,No. of Owners,Location,Price
19519,2020.0,Jeep,Jeep Compass 1.4 LIMITED PLUS AT,Petrol,AUTOMATIC,NaN,"52,301",1st,Pune,1988699.0
10435,2020.0,Mercedes-benz,Mercedes-benz C-class Progressive C 200,Petrol,Manual,1950.0,"18,000",First,New Delhi,4700000.0
4778,2018.0,Hyundai,Hyundai I20 Active 1.2 S,Petrol,Manual,1197.0,"53,647",First,Kolkata,590000.0
2384,2018.0,Hyundai,Hyundai Creta 1.6 Crdi At Sx Plus,Diesel,Automatic,1582.0,"36,000",First,Bangalore,1499000.0
19550,2012.0,Maruti,Maruti Wagon R 1.0 VXI,Petrol,MANUAL,NaN,"25,674",2nd,Pune,336999.0
...,...,...,...,...,...,...,...,...,...,...
19384,2018.0,Tata,Tata NEXON XZA+ 1.5,Diesel,AUTOMATIC,NaN,"13,704",2nd,Pune,1027299.0
13964,2016.0,Maruti,Maruti Baleno Delta Cvt,Petrol,Manual,1197.0,"26,700",First,Gurgaon,675000.0
2283,2009.0,Toyota,Toyota Corolla Altis G,Petrol,Manual,1798.0,"80,000",First,Bangalore,460000.0
4639,2014.0,Honda,Honda Amaze Ex I-vtech,Petrol,Manual,1198.0,"25,000",First,Kolkata,325000.0


In [118]:
df.reset_index(inplace=True,drop=True)
df

,Year,Brand,Model,Fuel Type,Transmission,Engine Capacity,Kms Driven,No. of Owners,Location,Price
0,2020.0,Jeep,Jeep Compass 1.4 LIMITED PLUS AT,Petrol,AUTOMATIC,NaN,"52,301",1st,Pune,1988699.0
1,2020.0,Mercedes-benz,Mercedes-benz C-class Progressive C 200,Petrol,Manual,1950.0,"18,000",First,New Delhi,4700000.0
2,2018.0,Hyundai,Hyundai I20 Active 1.2 S,Petrol,Manual,1197.0,"53,647",First,Kolkata,590000.0
3,2018.0,Hyundai,Hyundai Creta 1.6 Crdi At Sx Plus,Diesel,Automatic,1582.0,"36,000",First,Bangalore,1499000.0
4,2012.0,Maruti,Maruti Wagon R 1.0 VXI,Petrol,MANUAL,NaN,"25,674",2nd,Pune,336999.0
...,...,...,...,...,...,...,...,...,...,...
20906,2018.0,Tata,Tata NEXON XZA+ 1.5,Diesel,AUTOMATIC,NaN,"13,704",2nd,Pune,1027299.0
20907,2016.0,Maruti,Maruti Baleno Delta Cvt,Petrol,Manual,1197.0,"26,700",First,Gurgaon,675000.0
20908,2009.0,Toyota,Toyota Corolla Altis G,Petrol,Manual,1798.0,"80,000",First,Bangalore,460000.0
20909,2014.0,Honda,Honda Amaze Ex I-vtech,Petrol,Manual,1198.0,"25,000",First,Kolkata,325000.0


In [119]:
df.to_excel('Used_Car_Dataset.xlsx')